In [5]:
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility
import time
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Lambda
from keras.layers import Conv1D,MaxPooling1D, Flatten

from keras.utils import multi_gpu_model
from keras import backend as K
from sklearn.cross_validation import train_test_split
import matplotlib
matplotlib.use('agg')
import pylab as plt
import pandas as pd
from keras.utils.np_utils import to_categorical
from sklearn import preprocessing as pp
from keras.utils import np_utils
ss = pp.StandardScaler()
import h5py

X_train = np.load("20170827_mawilab_flow_000flags_nosvc_features.pkl")
Y_train = np.load("20170827_mawilab_flow_000flags_nosvc_labels.pkl")

# Test Set 4: 20170827_mawilab_flow_004
X_test4 = np.load("20170827_mawilab_flow_004flags_nosvc_features.pkl")
Y_test4 = np.load("20170827_mawilab_flow_004flags_nosvc_labels.pkl")
X_train = ss.fit_transform(X_train)
X_test4 = ss.fit_transform(X_test4)


# reshape input to be [samples, time steps, features]
X_train = np.reshape(X_train, (X_train.shape[0],X_train.shape[1],1))
X_test4 = np.reshape(X_test4, (X_test4.shape[0],X_test4.shape[1],1))


In [6]:
cnn1d_1 = Sequential([Conv1D(64, 3, padding="same",input_shape=(15, 1)),
    Activation('relu'),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(128),
    Activation('relu'),
    Dropout(0.5),
    Dense(2),
    Activation('sigmoid'),
])
print(cnn1d_1.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_2 (Conv1D)            (None, 15, 64)            256       
_________________________________________________________________
activation_4 (Activation)    (None, 15, 64)            0         
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 7, 64)             0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 448)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               57472     
_________________________________________________________________
activation_5 (Activation)    (None, 128)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
__________

In [7]:
#cnn1d_1 = multi_gpu_model(cnn1d_1, gpus=8)
cnn1d_1.compile(loss="binary_crossentropy", optimizer="adam",metrics=['accuracy'])
# train
start_time = time.time()
history = cnn1d_1.fit(X_train, Y_train, batch_size=64, validation_data=(X_test4, Y_test4) ,epochs=10)
end_time = time.time() 
print("Total time taken to train the Training model is", (end_time - start_time))
# serialize model to JSON
model_json = cnn1d_1.to_json()
with open("20170827_mawilab_flow_000.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
cnn1d_1.save_weights("20170827_mawilab_flow_000.h5")
print("Saved model to disk")
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
plt.savefig('Acc_Valaccuaracy00_04.png')

Train on 38361 samples, validate on 42599 samples
Epoch 1/10
38361/38361 [==============================] - 11s 274us/step - loss: 0.4807 - acc: 0.7038 - val_loss: 0.4748 - val_acc: 0.7257
Epoch 2/10
38361/38361 [==============================] - 9s 230us/step - loss: 0.4634 - acc: 0.7168 - val_loss: 0.4700 - val_acc: 0.7248
Epoch 3/10
38361/38361 [==============================] - 9s 229us/step - loss: 0.4599 - acc: 0.7189 - val_loss: 0.4686 - val_acc: 0.7303
Epoch 4/10
38361/38361 [==============================] - 9s 228us/step - loss: 0.4561 - acc: 0.7247 - val_loss: 0.4669 - val_acc: 0.7361
Epoch 5/10
38361/38361 [==============================] - 9s 226us/step - loss: 0.4537 - acc: 0.7260 - val_loss: 0.4618 - val_acc: 0.7345
Epoch 6/10
38361/38361 [==============================] - 9s 225us/step - loss: 0.4518 - acc: 0.7275 - val_loss: 0.4637 - val_acc: 0.7343
Epoch 7/10
38361/38361 [==============================] - 9s 230us/step - loss: 0.4511 - acc: 0.7294 - val_loss: 0.4681 -

In [8]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import f1_score
y_pred = cnn1d_1.predict(X_test4)
print(y_pred)
y_pred = np.argmax(y_pred, axis=1)
print(y_pred)

y_pred = cnn1d_1.predict_classes(X_test4)
print(y_pred)

p=cnn1d_1.predict_proba(X_test4)

target_names = ['normal', 'anomaly']
print(classification_report(np.argmax(Y_test4, axis=1), y_pred, target_names=target_names, digits=4))
print(confusion_matrix(np.argmax(Y_test4, axis=1), y_pred))
Y_test4 = np.argmax(Y_test4, axis=1)

print(f1_score(Y_test4, y_pred, average='macro'))

print(f1_score(Y_test4, y_pred, average='micro'))

print(f1_score(Y_test4, y_pred, average='weighted'))

print(f1_score(Y_test4, y_pred, average=None))

[[0.49133295 0.5085724 ]
 [0.00211486 0.9978756 ]
 [0.48837224 0.51152885]
 ...
 [0.49133295 0.5085724 ]
 [0.13596667 0.86297464]
 [0.62633777 0.37177184]]
[1 1 1 ... 1 1 0]
[1 1 1 ... 1 1 0]
             precision    recall  f1-score   support

     normal     0.5278    0.3003    0.3828     12411
    anomaly     0.7556    0.8895    0.8171     30188

avg / total     0.6892    0.7179    0.6906     42599

[[ 3727  8684]
 [ 3335 26853]]
0.5999592057876042
0.7178572266954624
0.6905876377550433
[0.38278642 0.81713199]
